In [ ]:
import jsonimport json
with open("outputs/buckets_07-24.json","r") as file:
  buckets = json.load(file)
bucket_map={action:bucket for bucket in buckets for action in buckets[bucket]}
bucket_names = list(buckets.keys())
with open("outputs/buckets_07-24.json","r") as file:
  buckets = json.load(file)
bucket_names = list(buckets.keys())

In [ ]:
import pandas as pd
datasets = [f"data/US/{term*2+2009-222}-{term*2+2010-222}_{term}th_Congress/csv/history.csv" for term in range(111,120)]
history_df_by_term=[pd.read_csv(ds) for ds in datasets]
history_df=pd.concat([pd.read_csv(dataset) for dataset in datasets])
# with open("./outputs/bucket_names_output.jsonl", "r") as file:
#   for line in file:
#     if line.strip():  # Skip empty lines
#       response = json.loads(line)
#       llm_buckets.append(response["response"]["body"]["output"][0]["content"][0]["text"])
      
# history_df["llm_bucket"] = history_df["bucket"].map(dict(zip(bucket_names, llm_buckets)))

# Add next bill_id column to compare
history_df["bucket"]=history_df["action"].apply(lambda action:bucket_map[action])
history_df["next_bill_id"]=history_df["bill_id"].shift(-1)
history_df["next_bucket"]=history_df["bucket"].shift(-1)
history_df["next_bucket"]=history_df.apply(lambda row:row["next_bucket"] if row["next_bill_id"]==row["bill_id"] else None,1)
for h in history_df_by_term:
  h["bucket"]=h["action"].apply(lambda action:bucket_map[action])
  h["next_bill_id"]=h["bill_id"].shift(-1)
  h["next_bucket"]=h["bucket"].shift(-1)
  h["next_bucket"]=h.apply(lambda row:row["next_bucket"] if row["next_bill_id"]==row["bill_id"] else None,1)

# history_df["next_bucket_llm"]=history_df["llm_bucket"].shift(-1)
# history_df["next_llm_bucket"]=history_df.apply(lambda row:row["next_bucket_llm"] if row["next_bill_id"]==row["bill_id"] else None,1)





# display(history_df)
# for i,row in history_df.iterrows():
#   print(row.bucket)
# for action in history_df:
#   history_df

In [ ]:
with open("outputs/bucket_names_07-24.txt","w") as file:
  for name in bucket_names:
    file.write(name)
    file.write("\n")

In [ ]:
introduced_in_senate=["Introduced in the Senate and read twice and ordered held at desk by unanimous consent. (Actions considered to have occured on 10/29/19 pursuant to the order of 11/12/19).",
"Introduced in the Senate, read the first time.",
"Introduced in the Senate, read twice, considered, read the third time, and passed without amendment and with a preamble by Unanimous Consent. (consideration: CR S1258-1260; text: CR S1259)",
"Introduced in the Senate, read twice, considered, read the third time, and passed without amendment by Voice Vote. (consideration: CR S1281-1282; text: CR S1281-1282)",
"Introduced in the Senate, read twice.",
"Introduced in the Senate.",
"Introduced in the Senate. Pursuant to Public Law 117-71, Ordered Placed on Senate Legislative Calendar under General Orders. Calendar No. 184.",
"Introduced in the Senate. Read the first time. Placed on Senate Legislative Calendar under Read the First Time.",
"Introduced in the Senate. Read twice. Placed on Senate Legislative Calendar under General Orders. Calendar No. 223 pursuant to Sec. 1002 of Public Law 113-46. (text of measure as introduced: CR S7580)",
"Introduced in the Senate. Read twice. Placed on Senate Legislative Calendar under General Orders. Calendar No. 350.",
"Introduced in the Senate. Read twice. Pursuant to the order of 2/14/2013 Placed on Senate Legislative Calendar under General Orders. Calendar No. 18."]
print(len(history_df[history_df['bucket'].isin(introduced_in_senate)]))

In [ ]:
bill_groups = {bill_id: group for bill_id, group in history_df.groupby('bill_id')}
display(bill_groups)
-- bills_passed = {bill_id: bill_groups[bill_id] for bill_id in history_df.groupby("bill_id") }

In [ ]:
display(history_df_by_term[1])

In [ ]:
# Constructing the graph
import networkx as nx
def df_to_graph(df):
  g_dict={}
  for bucket1 in bucket_names:
    g_dict[bucket1]={}
    for bucket2 in bucket_names:
      # print(bucket1,bucket2)
      g_dict[bucket1][bucket2]=0
    g_dict[bucket1][None]=0
  for i, row in df.iterrows():
    # print(row["bucket"])
    # print(row["next_bucket"])
    g_dict[row["bucket"]][row["next_bucket"]]+=1
  g = nx.DiGraph()

  for bucket in bucket_names:
    g.add_node(bucket)
  max_weight=0
  for bucket1 in bucket_names:
    for bucket2 in bucket_names:
      # if bucket1==bucket2:
      #   continue
      if g_dict[bucket1][bucket2]!=0:
        g.add_edge(bucket1,bucket2,weight=g_dict[bucket1][bucket2])
  # for name in bucket_names:
  #   if g_dict[name][name]!=0:
  #     g.add_edge(name,name+"_2",weight=g_dict[name][name])
  return g

g_overall = df_to_graph(history_df)
g_by_term = [df_to_graph(h) for h in history_df_by_term]
# for bucket1 in bucket_names:
#   for bucket2 in bucket_names:
#     g_dict[bucket1][bucket2]+=1
# for bucket1 in bucket_names:
#   for bucket2 in bucket_names:
#     g.
# g_tuples = {bucket:g_dict[bucket].items() for bucket in bucket_names}
# display(g_tuples)

In [ ]:
print(max([(deg,node) for node,deg in g.in_degree(weight=None) ]))

In [ ]:
print(inv_bucket_map['Placed on the House Calendar, Calendar No. 1.'])

In [ ]:
def bucket_names_repeated(times):
  return [name for name in bucket_names if len(inv_bucket_map[name])>=times]

In [ ]:
display([len(sorted(list(nx.strongly_connected_components(g)),key=len,reverse=True)[0]) for g in g_by_term])

In [ ]:
introduced_in_senate=["Introduced in the Senate and read twice and ordered held at desk by unanimous consent. (Actions considered to have occured on 10/29/19 pursuant to the order of 11/12/19).",
"Introduced in the Senate, read the first time.",
"Introduced in the Senate, read twice, considered, read the third time, and passed without amendment and with a preamble by Unanimous Consent. (consideration: CR S1258-1260; text: CR S1259)",
"Introduced in the Senate, read twice, considered, read the third time, and passed without amendment by Voice Vote. (consideration: CR S1281-1282; text: CR S1281-1282)",
"Introduced in the Senate, read twice.",
"Introduced in the Senate.",
"Introduced in the Senate. Pursuant to Public Law 117-71, Ordered Placed on Senate Legislative Calendar under General Orders. Calendar No. 184.",
"Introduced in the Senate. Read the first time. Placed on Senate Legislative Calendar under Read the First Time.",
"Introduced in the Senate. Read twice. Placed on Senate Legislative Calendar under General Orders. Calendar No. 223 pursuant to Sec. 1002 of Public Law 113-46. (text of measure as introduced: CR S7580)",
"Introduced in the Senate. Read twice. Placed on Senate Legislative Calendar under General Orders. Calendar No. 350.",
"Introduced in the Senate. Read twice. Pursuant to the order of 2/14/2013 Placed on Senate Legislative Calendar under General Orders. Calendar No. 18."]

In [ ]:
print(dict(g_by_term[2].in_degree())["Became Private Law No: 111-2."])

In [ ]:
for g in g_by_term:
  centr=nx.betweenness_centrality_subset(sources=bucket_names,targets=["Signed by President."],G=g,weight="weight")
  # display(centr["Pursuant to the provisions of H. Con. Res. 104, enrollment corrections on H.R. 1892 have been made."])
  # centr=nx.betweenness_centrality_subset(sources=introduced_in_senate+["Introduced in House"]+["Referred to the Committee on Agriculture, and in addition to the Committee on Appropriations, for a period to be subsequently determined by the Speaker, in each case for consideration of such provisions as fall within the jurisdiction of the committee concerned."],targets=["Became Private Law No: 111-2."],G=g,weight="weight")
  display(sorted([(centr[x],x) for x in centr if centr[x]>0],reverse=True))

In [ ]:
# centr=nx.betweenness_centrality_subset(sources=bucket_names,targets=["Became Private Law No: 111-2."],G=g,weight="weight")

In [ ]:
print(inv_bucket_map["Introduced in House"])

In [ ]:
out_degs= dict(g.out_degree(weight="weight"))
# display(out_degs)
common_nodes=[x for x in out_degs if out_degs[x]>=10]
common_node_g=g.subgraph(common_nodes)
# display(out_degs)
connected_subgraph=g.subgraph(sorted(nx.strongly_connected_components(common_node_g),key=len,reverse=True)[0])
# display(connected_subgraph)
# connected_subgraph_common_nodes=connected_subgraph.subgraph(sorted(connected_subgraph.out_degree())

In [ ]:
display(bucket_names)

In [ ]:
min_cut=list(nx.minimum_edge_cut(connected_subgraph))
print(len(min_cut))
# print(min_cut)
display(min_cut,connected_subgraph.get_edge_data(min_cut[0],min_cut[1]))

In [ ]:
print(inv_bucket_map["Committee Consideration and Mark-up Session Held"])

In [ ]:
display(list(g.predecessors('the motion to favorably report was withdrawn by unanimous consent')))

In [ ]:
min_cut=nx.is_strongly_connected(connected_subgraph)
display(min_cut)

In [ ]:
display([len(x) for x in nx.strongly_connected_components(g)])

In [ ]:
print(nx.to_pandas_edgelist(g))

In [ ]:
import networkx as nx
components=nx.strongly_connected_components(g)
display(len([comp for comp in components][0]))
# nx.draw(g)
